#Data Cleaning

Dataset:Brazilian E-Commerce Public Dataset by Olist

Link:https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce

Loading Datasets


In [2]:
import pandas as pd
import numpy as np
customers   = pd.read_csv('olist_customers_dataset.csv')
geolocation = pd.read_csv('olist_geolocation_dataset.csv')
orders      = pd.read_csv('olist_orders_dataset.csv')
order_items = pd.read_csv('olist_order_items_dataset.csv')
payments    = pd.read_csv('olist_order_payments_dataset.csv')
reviews     = pd.read_csv('olist_order_reviews_dataset.csv')
products    = pd.read_csv('olist_products_dataset.csv')
sellers     = pd.read_csv('olist_sellers_dataset.csv')
category_map = pd.read_csv('product_category_name_translation.csv')

print("✅ All datasets loaded successfully!")


✅ All datasets loaded successfully!


 Basic structure: shape, head, duplicate checking

In [3]:
datasets = {
    "customers": customers,
    "orders": orders,
    "order_items": order_items,
    "payments": payments,
    "reviews": reviews,
    "products": products,
    "sellers": sellers
}

for name, df in datasets.items():
    print(f"\n{name.upper()} ➤ shape: {df.shape}, duplicates: {df.duplicated().sum()}")
    df.drop_duplicates(inplace=True)



CUSTOMERS ➤ shape: (99441, 5), duplicates: 0

ORDERS ➤ shape: (99441, 8), duplicates: 0

ORDER_ITEMS ➤ shape: (112650, 7), duplicates: 0

PAYMENTS ➤ shape: (103886, 5), duplicates: 0

REVIEWS ➤ shape: (99224, 7), duplicates: 0

PRODUCTS ➤ shape: (32951, 9), duplicates: 0

SELLERS ➤ shape: (3095, 4), duplicates: 0


Convert datetime fields to proper format

In [4]:
date_cols = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

for col in date_cols:
    orders[col] = pd.to_datetime(orders[col], errors='coerce')


 Merge key tables to build a master dataframe

In [6]:
df = (orders
      .merge(order_items, on='order_id', how='left')
      .merge(payments, on='order_id', how='left')
      .merge(reviews[['order_id', 'review_score']], on='order_id', how='left')
      .merge(customers[['customer_id','customer_unique_id','customer_city','customer_state']], on='customer_id', how='left')
      .merge(products[['product_id','product_category_name']], on='product_id', how='left')
      )
df = df.merge(category_map, on='product_category_name', how='left')


Create new useful columns

In [7]:
df['delivery_time_days'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.days
df['delay_vs_estimate_days'] = (df['order_delivered_customer_date'] - df['order_estimated_delivery_date']).dt.days
df['is_late'] = df['delay_vs_estimate_days'] > 0

df['order_month'] = df['order_purchase_timestamp'].dt.to_period('M')

Handle missing values

In [8]:
df['review_score'] = df['review_score'].fillna(df['review_score'].median())

# Example: replace missing delivery dates where order was never delivered
df['delivery_time_days'] = df['delivery_time_days'].fillna(-1)


Save cleaned dataset

In [10]:
df.to_csv('clean_merged_data.csv', index=False)
print("✅ cleaned master file saved as clean_merged_data.csv")
print("\nSample cleaned data:")
print(df.head())

✅ cleaned master file saved as clean_merged_data.csv

Sample cleaned data:
                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
2  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
3  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
4  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   

  order_status order_purchase_timestamp   order_approved_at  \
0    delivered      2017-10-02 10:56:33 2017-10-02 11:07:15   
1    delivered      2017-10-02 10:56:33 2017-10-02 11:07:15   
2    delivered      2017-10-02 10:56:33 2017-10-02 11:07:15   
3    delivered      2018-07-24 20:41:37 2018-07-26 03:24:27   
4    delivered      2018-08-08 08:38:49 2018-08-08 08:55:23   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-04 19:55:00           20